In [1]:
# Julia 1.8.4
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.add("CUDA")
Pkg.add("Glob")
Pkg.add("Flux")
Pkg.add("NIfTI")
Pkg.add("Images")
Pkg.add("FastAI")
Pkg.add("Metalhead")
Pkg.add("FastVision")
Pkg.add("CairoMakie")
Pkg.add("StaticArrays")
Pkg.add("MLDataPattern")
Pkg.add("BenchmarkTools")
Pkg.add("ChainRulesCore")
Pkg.add("BSON")
Pkg.add("DataFrames")
Pkg.add("CSV")
Pkg.add(url = "https://github.com/Dale-Black/ComputerVisionMetrics.jl")
Pkg.add(url="https://github.com/Dale-Black/DistanceTransforms.jl", rev="wenbo")

  Activating project at `c:\Users\wenbl13\Desktop\Wenbo_Timing`


    Updating registry at `C:\Users\wenbl13\.julia\registries\General.toml`


   Resolving package versions...

  No Changes to `C:\Users\wenbl13\Desktop\Wenbo_Timing\Project.toml`
  No Changes to `C:\Users\wenbl13\Desktop\Wenbo_Timing\Manifest.toml`


   Resolving package versions...


...

package versions...


  No Changes to `C:\Users\wenbl13\Desktop\Wenbo_Timing\Project.toml`
  No Changes to `C:\Users\wenbl13\Desktop\Wenbo_Timing\Manifest.toml`

    Updating git-repo `https://github.com/Dale-Black/DistanceTransforms.jl`


   Resolving package versions...


  No Changes to `C:\Users\wenbl13\Desktop\Wenbo_Timing\Project.toml`
  No Changes to `C:\Users\wenbl13\Desktop\Wenbo_Timing\Manifest.toml`


In [2]:
using CUDA
using Glob
using Dates
using NIfTI
using Images
using Statistics
using StaticArrays
using MLDataPattern
using BenchmarkTools
using ChainRulesCore
using DistanceTransforms
using DataFrames
using CSV
using ComputerVisionMetrics
using FastAI, FastVision, Flux, Metalhead
import CairoMakie; CairoMakie.activate!(type="png")

# Load and prepare data

In [3]:
data_dir = raw"C:\Users\wenbl13\Desktop\Ashwin-Timing\distance-transforms\Task02_Heart"

"C:\\Users\\wenbl13\\Desktop\\Ashwin-Timing\\distance-transforms\\Task02_Heart"

In [28]:
function loadfn_label(p)
    a = NIfTI.niread(string(p)).raw
    convert_a = convert(Array{UInt8}, a)
    convert_a = convert_a .+ 1
    return convert_a
end

function loadfn_image(p)
    a = NIfTI.niread(string(p)).raw
    convert_a = convert(Array{Float32}, a)
    convert_a = convert_a / max(convert_a...)
    return convert_a
end

images(dir) = mapobs(loadfn_image, Glob.glob("*.nii*", dir))
masks(dir) =  mapobs(loadfn_label, Glob.glob("*.nii*", dir))
pre_data = (
    images(joinpath(data_dir, "imagesTr")),
    masks(joinpath(data_dir, "labelsTr")),
);

image_size = (96, 96, 96)

function presize(files)
    container_images = Array{Float32,4}(undef, image_size..., numobs(files))
    container_masks = Array{Int64,4}(undef, image_size..., numobs(files))
    for i in 1:numobs(files)
        image, mask = FastAI.getobs(files, i)
        img = imresize(image, image_size)
        msk = round.(imresize(mask, image_size))
        container_images[:, :, :, i] = img
        container_masks[:, :, :, i] = msk
    end
    return container_images, container_masks
end

img_container, mask_container = presize(pre_data)
data_resized = (img_container,mask_container);

a, b = FastVision.imagedatasetstats(img_container, Gray{N0f8}) 
means, stds = SVector{1, Float32}(a[1]), SVector{1, Float32}(b[1])

task = SupervisedTask(
    (FastVision.Image{3}(), Mask{3}(1:2)),
    (
        ProjectiveTransforms((image_size)),
        ImagePreprocessing(means = means, stds = stds, C = Gray{N0f8}),
        FastAI.OneHot()
    )
)

train_files, val_files = MLDataPattern.splitobs(data_resized, 0.8);

Progress:  10%|█████                                    |  ETA: 0:00:02

Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


In [32]:
batch_size = 4 
tdl, vdl = FastAI.taskdataloaders(train_files, val_files, task, batch_size);

# Get metrics

In [37]:
dice_metric_of_dice_model_train_set = []
hd_metric_of_dice_model_train_set = []
dice_metric_of_HD_Dice_model_train_set = []
hd_metric_of_HD_Dice_model_train_set = []

dice_metric_of_dice_model_valid_set = []
hd_metric_of_dice_model_valid_set = []
dice_metric_of_HD_Dice_model_valid_set = []
hd_metric_of_HD_Dice_model_valid_set = []

for epoch_idx = 10 : 10 : 500
    print("$epoch_idx...\t")
    # Load models
    DICE_model_path = string("savedmodels/bigger_NN_0.001_Dice_", epoch_idx, ".jld2")
    HD_DICE_model_path = string("savedmodels/bigger_NN_0.001_HD_Dice_", epoch_idx, ".jld2")
    _ , model_Dice = loadtaskmodel(DICE_model_path)
    _ , model_HD_Dice = loadtaskmodel(HD_DICE_model_path)
    model_Dice = model_Dice |> gpu
    model_HD_Dice = model_HD_Dice |> gpu
    _ = nothing
    # Get metrics

    # Traverse train dataloader
    curr_dice_metric_of_dice_model = []
    curr_hd_metric_of_dice_model = []
    curr_dice_metric_of_HD_Dice_model = []
    curr_hd_metric_of_HD_Dice_model = []
    for (_img, _mask) in tdl
        mask = _mask[:,:,:,2,:] .>= 0.5
        img = _img |> gpu

        # Predictions
        pred_mask_DICE = (model_Dice(img) |> cpu)[:,:,:,2,:] .>= 0.5
        pred_mask_HD_DICE = (model_HD_Dice(img) |> cpu)[:,:,:,2,:] .>= 0.5

        curr_curr_dice_metric_of_dice_model = Array{Any}(undef, 4)
        curr_curr_hd_metric_of_dice_model = Array{Any}(undef, 4)
        curr_curr_dice_metric_of_HD_Dice_model = Array{Any}(undef, 4)
        curr_curr_hd_metric_of_HD_Dice_model = Array{Any}(undef, 4)

        Threads.@threads for job_idx = 1:16
            batch_idx = ceil(Int, job_idx / 4)
            job = job_idx % 4

            # Dice metric for Dice model
            job == 0 && (curr_curr_dice_metric_of_dice_model = ComputerVisionMetrics.dice(pred_mask_DICE[:,:,:,batch_idx], mask[:,:,:,batch_idx]))

            # HD metric for Dice model
            job == 1 && (curr_curr_hd_metric_of_dice_model = ComputerVisionMetrics.hausdorff(pred_mask_DICE[:,:,:,batch_idx], mask[:,:,:,batch_idx]))

            # Dice metric for HD+Dice model
            job == 2 && (curr_curr_dice_metric_of_HD_Dice_model = ComputerVisionMetrics.dice(pred_mask_HD_DICE[:,:,:,batch_idx], mask[:,:,:,batch_idx]))

            # HD metric for HD+Dice model
            job == 3 && (curr_curr_hd_metric_of_HD_Dice_model = ComputerVisionMetrics.hausdorff(pred_mask_HD_DICE[:,:,:,batch_idx], mask[:,:,:,batch_idx]))
        end
        # Record
        append!(curr_dice_metric_of_dice_model, curr_curr_dice_metric_of_dice_model)
        append!(curr_hd_metric_of_dice_model, curr_curr_hd_metric_of_dice_model)
        append!(curr_dice_metric_of_HD_Dice_model, curr_curr_dice_metric_of_HD_Dice_model)
        append!(curr_hd_metric_of_HD_Dice_model, curr_curr_hd_metric_of_HD_Dice_model)
    end
    push!(dice_metric_of_dice_model_train_set, mean(curr_dice_metric_of_dice_model))
    push!(hd_metric_of_dice_model_train_set, mean(curr_hd_metric_of_dice_model))
    push!(dice_metric_of_HD_Dice_model_train_set, mean(curr_dice_metric_of_HD_Dice_model))
    push!(hd_metric_of_HD_Dice_model_train_set, mean(curr_hd_metric_of_HD_Dice_model))


    curr_dice_metric_of_dice_model = []
    curr_hd_metric_of_dice_model = []
    curr_dice_metric_of_HD_Dice_model = []
    curr_hd_metric_of_HD_Dice_model = []
    # Traverse valid dataloader
    for (_img, _mask) in vdl
        mask = _mask[:,:,:,2,:] .>= 0.5
        img = _img |> gpu

        # Predictions
        pred_mask_DICE = (model_Dice(img) |> cpu)[:,:,:,2,:] .>= 0.5
        pred_mask_HD_DICE = (model_HD_Dice(img) |> cpu)[:,:,:,2,:] .>= 0.5

        curr_curr_dice_metric_of_dice_model = Array{Any}(undef, 4)
        curr_curr_hd_metric_of_dice_model = Array{Any}(undef, 4)
        curr_curr_dice_metric_of_HD_Dice_model = Array{Any}(undef, 4)
        curr_curr_hd_metric_of_HD_Dice_model = Array{Any}(undef, 4)

        Threads.@threads for job_idx = 1:16
            batch_idx = ceil(Int, job_idx / 4)
            job = job_idx % 4

            # Dice metric for Dice model
            job == 0 && (curr_curr_dice_metric_of_dice_model = ComputerVisionMetrics.dice(pred_mask_DICE[:,:,:,batch_idx], mask[:,:,:,batch_idx]))

            # HD metric for Dice model
            job == 1 && (curr_curr_hd_metric_of_dice_model = ComputerVisionMetrics.hausdorff(pred_mask_DICE[:,:,:,batch_idx], mask[:,:,:,batch_idx]))

            # Dice metric for HD+Dice model
            job == 2 && (curr_curr_dice_metric_of_HD_Dice_model = ComputerVisionMetrics.dice(pred_mask_HD_DICE[:,:,:,batch_idx], mask[:,:,:,batch_idx]))

            # HD metric for HD+Dice model
            job == 3 && (curr_curr_hd_metric_of_HD_Dice_model = ComputerVisionMetrics.hausdorff(pred_mask_HD_DICE[:,:,:,batch_idx], mask[:,:,:,batch_idx]))
        end
        # Record
        append!(curr_dice_metric_of_dice_model, curr_curr_dice_metric_of_dice_model)
        append!(curr_hd_metric_of_dice_model, curr_curr_hd_metric_of_dice_model)
        append!(curr_dice_metric_of_HD_Dice_model, curr_curr_dice_metric_of_HD_Dice_model)
        append!(curr_hd_metric_of_HD_Dice_model, curr_curr_hd_metric_of_HD_Dice_model)
    end
    push!(dice_metric_of_dice_model_valid_set, mean(curr_dice_metric_of_dice_model))
    push!(hd_metric_of_dice_model_valid_set, mean(curr_hd_metric_of_dice_model))
    push!(dice_metric_of_HD_Dice_model_valid_set, mean(curr_dice_metric_of_HD_Dice_model))
    push!(hd_metric_of_HD_Dice_model_valid_set, mean(curr_hd_metric_of_HD_Dice_model))
    pred_mask_DICE = nothing
    pred_mask_HD_DICE = nothing
    curr_dice_metric_of_dice_model = nothing
    curr_hd_metric_of_dice_model = nothing
    curr_dice_metric_of_HD_Dice_model = nothing
    curr_hd_metric_of_HD_Dice_model = nothing
    GC.gc(true)
    CUDA.reclaim()
end
println()

10...	

┌ Warning: Number of observations less than batch-size, decreasing the batch-size to 4
└ @ MLUtils C:\Users\wenbl13\.julia\packages\MLUtils\Th9Y3\src\batchview.jl:95


20...	

┌ Warning: Number of observations less than batch-size, decreasing the batch-size to 4
└ @ MLUtils C:\Users\wenbl13\.julia\packages\MLUtils\Th9Y3\src\batchview.jl:95


30...	

...

┌ Warning: Number of observations less than batch-size, decreasing the batch-size to 4
└ @ MLUtils C:\Users\wenbl13\.julia\packages\MLUtils\Th9Y3\src\batchview.jl:95


490...	

┌ Warning: Number of observations less than batch-size, decreasing the batch-size to 4
└ @ MLUtils C:\Users\wenbl13\.julia\packages\MLUtils\Th9Y3\src\batchview.jl:95


500...	

┌ Warning: Number of observations less than batch-size, decreasing the batch-size to 4
└ @ MLUtils C:\Users\wenbl13\.julia\packages\MLUtils\Th9Y3\src\batchview.jl:95


# Record

In [38]:
epoch_idx = collect(10 : 10 : 500)
df_metrics = DataFrame(epoch_idx = epoch_idx,
    dice_metric_of_dice_model_train_set = dice_metric_of_dice_model_train_set,
    hd_metric_of_dice_model_train_set = hd_metric_of_dice_model_train_set,
    dice_metric_of_HD_Dice_model_train_set = dice_metric_of_HD_Dice_model_train_set,
    hd_metric_of_HD_Dice_model_train_set = hd_metric_of_HD_Dice_model_train_set,

    dice_metric_of_dice_model_valid_set = dice_metric_of_dice_model_valid_set,
    hd_metric_of_dice_model_valid_set = hd_metric_of_dice_model_valid_set,
    dice_metric_of_HD_Dice_model_valid_set = dice_metric_of_HD_Dice_model_valid_set,
    hd_metric_of_HD_Dice_model_valid_set = hd_metric_of_HD_Dice_model_valid_set)
CSV.write("CSVs/metrics_Task02_Heart_lr_is_0.001.csv", df_metrics)

"CSVs/metrics_Task02_Heart_lr_is_0.001.csv"